In [2]:
pip install -r requirements.txt

  Using cached numpy-2.3.5-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached pandas-2.3.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached scikit_learn-1.7.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.3-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.1-cp311-cp311-win_amd64.whl.metadata (114 kB)
  Using cached kiwisolver-1.4.9-cp311-cp311-win_amd64.whl.metadata (6.4 kB)
  Using cached pillow-12.0.0-cp311-cp311-win_amd64.whl.metadata (9.0 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (py


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import numpy as np
import pandas as pd

In [10]:
%pip install textblob

     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ------------------ ------------------- 20.5/41.5 kB 330.3 kB/s eta 0:00:01
     -------------------------------------  41.0/41.5 kB 495.5 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 401.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ------------------------------ -------- 491.5/624.3 kB 15.5 MB/s eta 0:00:01
   ---------------------------------------- 624.3/624.3 kB 9.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ----------------------------------- ---- 1.4/1.5 MB 28.5 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 19.0 MB/s eta 0:00:00
   -------


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from textblob import TextBlob

In [18]:

df=pd.read_csv("datasets/data.csv")
df["sentiment"] = df["content"].fillna("").apply(lambda x: TextBlob(str(x)).sentiment.polarity)
print("Sentiment calculated.")

Sentiment calculated.


#### GeoRisk Signal Index (GRSI)

A lightweight numeric score generated daily from news sentiment + tension + topic classification.

**For each day:**

**GRSI** = (Weighted Negative Sentiment) + (Conflict Topic Weight) + (Tension Count)

In [21]:
df["country"] = df["category"].fillna("Unknown")
df = df[df["country"] != "Unknown"]
conflict_terms = [
    "war","attack","strike","conflict","violence","crisis","explosion","terror",
    "missile","border","sanction","tension","dispute","military","dead","killed"
]

def conflict_score(text):
    text = str(text).lower()
    return sum(text.count(word) for word in conflict_terms)

df["conflict_count"] = df["content"].apply(conflict_score)
w1, w2, w3 = 0.5, 0.3, 0.2


In [22]:
country_stats = df.groupby("country").agg({
    "sentiment": "mean",
    "conflict_count": "sum",
    "article_id": "count"
}).reset_index()

country_stats["negative_sentiment"] = country_stats["sentiment"].apply(lambda x: abs(x) if x < 0 else 0)

country_stats["GRSI"] = (
    w1 * country_stats["negative_sentiment"] +
    w2 * country_stats["conflict_count"] +
    w3 * country_stats["article_id"]
)

country_stats = country_stats.sort_values("GRSI", ascending=False)
country_stats.head()


,country,sentiment,conflict_count,article_id,negative_sentiment,GRSI
222,Stock,0.157353,115,3999,0.0,834.3
104,Health,0.079032,674,2594,0.0,721.0
230,Technology,0.124612,259,2371,0.0,551.9
80,Finance,0.100876,209,2402,0.0,543.1
167,News,0.079580,456,1852,0.0,507.2


In [23]:
company_country_map = {
    "TCS": "India", "Reliance": "India", "Adani": "India", "HDFC": "India",
    "Toyota": "Japan", "Honda": "Japan", "Sony": "Japan", "Nintendo": "Japan",
    "Alibaba": "China", "Xiaomi": "China", "JD.com Inc": "China", "Tencent": "China"
}

company_risk = pd.DataFrame([
    {"company": comp, "country": country, "GRSI": country_stats.set_index("country").loc[country]["GRSI"]}
    for comp, country in company_country_map.items()
])

company_risk.to_csv("datasets/company_risk.csv", index=False)
country_stats.to_csv("datasets/country_GRSI.csv", index=False)

print("GRSI files saved.")
company_risk


GRSI files saved.


,company,country,GRSI
0,TCS,India,269.8
1,Reliance,India,269.8
2,Adani,India,269.8
3,HDFC,India,269.8
4,Toyota,Japan,72.9
5,Honda,Japan,72.9
6,Sony,Japan,72.9
7,Nintendo,Japan,72.9
8,Alibaba,China,130.8
9,Xiaomi,China,130.8
